<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Chocolate Gummy Bears](#Q2)
    - Perceptron
    - Multilayer Perceptron
4. [Keras MMP](#Q3)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:**
A basic unit that carries a single number.
- **Input Layer:**
Input layers are the features of the dataset.
- **Hidden Layer:**
Hidden layers are in-between the input and output that preform tasks on the neurons.
- **Output Layer:**
Output layer is the final layer of a neural net that delivers the result/s.
- **Activation:**
An activation function computs the neuron to determine the weights.
- **Backpropagation:**
A mechanism that tell the neural net if a prediction was correct or not. 

## 2. Chocolate Gummy Bears <a id="Q2"></a>

Right now, you're probably thinking, "yuck, who the hell would eat that?". Great question. Your candy company wants to know too. And you thought I was kidding about the [Chocolate Gummy Bears](https://nuts.com/chocolatessweets/gummies/gummy-bears/milk-gummy-bears.html?utm_source=google&utm_medium=cpc&adpos=1o1&gclid=Cj0KCQjwrfvsBRD7ARIsAKuDvMOZrysDku3jGuWaDqf9TrV3x5JLXt1eqnVhN0KM6fMcbA1nod3h8AwaAvWwEALw_wcB). 

Let's assume that a candy company has gone out and collected information on the types of Halloween candy kids ate. Our candy company wants to predict the eating behavior of witches, warlocks, and ghosts -- aka costumed kids. They shared a sample dataset with us. Each row represents a piece of candy that a costumed child was presented with during "trick" or "treat". We know if the candy was `chocolate` (or not chocolate) or `gummy` (or not gummy). Your goal is to predict if the costumed kid `ate` the piece of candy. 

If both chocolate and gummy equal one, you've got a chocolate gummy bear on your hands!?!?!
![Chocolate Gummy Bear](https://ed910ae2d60f0d25bcb8-80550f96b5feb12604f4f720bfefb46d.ssl.cf1.rackcdn.com/3fb630c04435b7b5-2leZuM7_-zoom.jpg)

In [2]:
import pandas as pd
candy = pd.read_csv('chocolate_gummy_bears.csv')

In [3]:
candy.head()

,chocolate,gummy,ate
0,0,1,1
1,1,0,1
2,0,1,1
3,0,0,0
4,1,1,0


In [4]:
candy.shape

(10000, 3)

### Perceptron

To make predictions on the `candy` dataframe. Build and train a Perceptron using numpy. Your target column is `ate` and your features: `chocolate` and `gummy`. Do not do any feature engineering. :P

Once you've trained your model, report your accuracy. You will not be able to achieve more than ~50% with the simple perceptron. Explain why you could not achieve a higher accuracy with the *simple perceptron* architecture, because it's possible to achieve ~95% accuracy on this dataset. Provide your answer in markdown (and *optional* data anlysis code) after your perceptron implementation. 

In [6]:
# Start your candy perceptron here

X = candy[['chocolate', 'gummy']].values
y = candy['ate'].values

In [36]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivate(x):
    sx = sigmoid(x)
    return( sx * (1-sx))

class Perceptron(object):
    
    def __init__(self, niter = 1):
        self.niter = niter
    

    def fit(self, X, y):
        
        inputs = X
        correct_outputs = y.reshape(-1,1)

        weights = 2 * np.random.random((X.shape[1],1)) - 1
       # bias = 2 * np.random.random((len(X),1)) - 1
        
        for i in range(self.niter):

            weighted_sum = np.dot(inputs, weights)# + bias

            activated_output = sigmoid(weighted_sum)

   
            error = correct_outputs - activated_output

            adjustments = error * sigmoid_derivate(activated_output)

  
            #bias += error
            weights += np.dot(inputs.T, adjustments)
        self.activated_output = activated_output
        self.weights = weights
        #self.bias = bias
        

    def predict(self, X):
        weighted_sum = np.dot(X, self.weights) #+ self.bias
        
        return sigmoid(weighted_sum)

In [37]:
nn = Perceptron()
nn.fit(X,y)

In [38]:
pred = nn.predict(X)

n_pred = []
for i in pred:
    n_pred.append(int(i))
from sklearn.metrics import accuracy_score as acs

acs(y, n_pred)



0.5

## 50% accuracy score

A simple percoptron is not able to preform well becuase it does not have a mechanism to update the weights effectively. Also this model does not implemtent bias witch prohibts the the model from learning the weights. If bias is implemtened, the accuracy score jumps to 75%.

### Multilayer Perceptron <a id="Q3"></a>

Using the sample candy dataset, implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. Your Multilayer Perceptron should be implemented in Numpy. 
Your network must have one hidden layer.

Once you've trained your model, report your accuracy. Explain why your MLP's performance is considerably better than your simple perceptron's on the candy dataset. 

In [45]:
class NeuralNetwork:
    def __init__(self):

        self.inputs = 2
        self.hiddenNodes = 3
        self.outputNodes = 1


        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):

        self.hidden_sum = np.dot(X, self.weights1)

        self.activated_hidden = self.sigmoid(self.hidden_sum)
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X,y,o):

        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        self.z2_error = self.o_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

In [49]:
nn = NeuralNetwork()


In [53]:
y.reshape(-1,1)

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [55]:
for i in range(10000):
    if (i in [a for a in range(7)]) or ((i + 1) % 5000 == 0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y.reshape(-1,1))
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y.reshape(-1,1) - nn.feed_forward(X)))))
    nn.train(X,y.reshape(-1,1))

+---------EPOCH 1---------+
Input: 
 [[0 1]
 [1 0]
 [0 1]
 ...
 [0 1]
 [0 1]
 [1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
Predicted Output: 
 [[0.49999985]
 [0.49999982]
 [0.49999985]
 ...
 [0.49999985]
 [0.49999985]
 [0.49999982]]
Loss: 
 0.20115007378461938
+---------EPOCH 2---------+
Input: 
 [[0 1]
 [1 0]
 [0 1]
 ...
 [0 1]
 [0 1]
 [1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
Predicted Output: 
 [[0.49999985]
 [0.49999982]
 [0.49999985]
 ...
 [0.49999985]
 [0.49999985]
 [0.49999982]]
Loss: 
 0.2011500737709538
+---------EPOCH 3---------+
Input: 
 [[0 1]
 [1 0]
 [0 1]
 ...
 [0 1]
 [0 1]
 [1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
Predicted Output: 
 [[0.49999985]
 [0.49999982]
 [0.49999985]
 ...
 [0.49999985]
 [0.49999985]
 [0.49999982]]
Loss: 
 0.20115007375729324
+---------EPOCH 4---------+
Input: 
 [[0 1]
 [1 0]
 [0 1]
 ...
 [0 1]
 [0 1]
 [1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
Predicted Output: 
 [[0.49999985]
 

In [58]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivate(x):
    sx = sigmoid(x)
    return( sx * (1-sx))

class Perceptron(object):
    
    def __init__(self, niter = 500):
        self.niter = niter
    

    def fit(self, X, y):
        
        inputs = X
        correct_outputs = y.reshape(-1,1)

        weights = 2 * np.random.random((X.shape[1],1)) - 1
        bias = 2 * np.random.random((len(X),1)) - 1
        
        for i in range(self.niter):

            weighted_sum = np.dot(inputs, weights) + bias

            activated_output = sigmoid(weighted_sum)

   
            error = correct_outputs - activated_output

            adjustments = error * sigmoid_derivate(activated_output)

  
            bias += error
            weights += np.dot(inputs.T, adjustments)
        self.activated_output = activated_output
        self.weights = weights
        self.bias = bias
        

    def predict(self, X):
        weighted_sum = np.dot(X, self.weights) + self.bias
        
        return sigmoid(weighted_sum)

In [60]:
nn = Perceptron()
nn.fit(X,y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [62]:
pred = nn.predict(X)
n_pred = []
for i in pred:
    n_pred.append(int(i))

acs(y,n_pred)

0.9859

## Backprop and bias leads to an accuracy score of +95%.

Backpropagation allows the neural net to learn from each mistake to better determine the weights.

P.S. Don't try candy gummy bears. They're disgusting. 

## 3. Keras MMP <a id="Q3"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV or RandomSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 3 parameters in order to get a 3 on this section.

## Four main hyperparameter focuses
- Number of layers
- Number of echos
- batch_size
- Number of nodes

In [63]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
270,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0
150,66,1,0,160,228,0,0,138,0,2.3,2,0,1,1
247,66,1,1,160,246,0,1,120,1,0.0,1,3,1,0
77,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
90,48,1,2,124,255,1,1,175,0,0.0,2,2,2,1


In [79]:
X = df.values[:,:-1]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-0.92271345,  0.68100522, -0.93851463, ...,  0.97635214,
        -0.71442887,  1.12302895],
       [ 1.28306308,  0.68100522, -0.93851463, ...,  0.97635214,
        -0.71442887, -2.14887271],
       [ 1.28306308,  0.68100522,  0.03203122, ..., -0.64911323,
         2.22410436, -2.14887271],
       ...,
       [-1.36386876,  0.68100522,  0.03203122, ...,  0.97635214,
        -0.71442887, -0.51292188],
       [-0.92271345,  0.68100522,  0.03203122, ...,  0.97635214,
        -0.71442887,  1.12302895],
       [-1.1432911 ,  0.68100522, -0.93851463, ..., -2.27457861,
        -0.71442887, -2.14887271]])

In [80]:
y = df.target.values

In [81]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def build_model():
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [85]:
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

model = KerasClassifier(build_fn=build_model, verbose=False, validation_split=0.10)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.6666666666666666 using {'batch_size': 10, 'epochs': 20}
Means: 0.6666666666666666, Stdev: 0.10550736937169447 with: {'batch_size': 10, 'epochs': 20}
Means: 0.5841584106286367, Stdev: 0.08440072713159942 with: {'batch_size': 20, 'epochs': 20}
Means: 0.5709570944309235, Stdev: 0.060675823753857584 with: {'batch_size': 40, 'epochs': 20}
Means: 0.528052826722463, Stdev: 0.009334740203990824 with: {'batch_size': 60, 'epochs': 20}
Means: 0.4785478711128235, Stdev: 0.08719369697853102 with: {'batch_size': 80, 'epochs': 20}
Means: 0.6204620401064554, Stdev: 0.12892426620220182 with: {'batch_size': 100, 'epochs': 20}


### Base line score of 66%

In [89]:
def build_model():
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=build_model, verbose=False, validation_split=0.10)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7887788812319437 using {'batch_size': 10, 'epochs': 20}
Means: 0.7887788812319437, Stdev: 0.03820407080172275 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7821782231330872, Stdev: 0.016168244308767975 with: {'batch_size': 20, 'epochs': 20}
Means: 0.6963696380456289, Stdev: 0.15640917850366245 with: {'batch_size': 40, 'epochs': 20}
Means: 0.6963696281115214, Stdev: 0.05382676587232076 with: {'batch_size': 60, 'epochs': 20}
Means: 0.49504950642585754, Stdev: 0.0820449031203073 with: {'batch_size': 80, 'epochs': 20}
Means: 0.6336633662382761, Stdev: 0.10876077367654916 with: {'batch_size': 100, 'epochs': 20}


### Added layers as the first for the first tuned parameter. Confirmed batch_size of 10. Model increased to 78% accuracy.

In [91]:
dir(grid_result)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',


In [95]:
grid_result.cv_results_

{'mean_fit_time': array([2.07969395, 1.75161465, 1.52807903, 1.50410398, 1.45364682,
        1.46441897]),
 'std_fit_time': array([0.15510651, 0.26054729, 0.19578866, 0.18349208, 0.26392152,
        0.17134057]),
 'mean_score_time': array([0.05069606, 0.05028526, 0.04780968, 0.04511476, 0.04577573,
        0.04668721]),
 'std_score_time': array([0.00265777, 0.00283199, 0.00419126, 0.00085364, 0.00402443,
        0.00371894]),
 'param_batch_size': masked_array(data=[10, 20, 40, 60, 80, 100],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[20, 20, 20, 20, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 10, 'epochs': 20},
  {'batch_size': 20, 'epochs': 20},
  {'batch_size': 40, 'epochs': 20},
  {'batch_size': 60, 'epochs': 20},
  {'batch_size': 80, 'epochs': 20},
  {'batch_size': 100, '

In [96]:
param_grid = {'batch_size': [10],
              'epochs': [20,30,45,60,80,120]}

model = KerasClassifier(build_fn=build_model, verbose=False, validation_split=0.10)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.8118811845779419 using {'batch_size': 10, 'epochs': 30}
Means: 0.7722772359848022, Stdev: 0.008084122154383987 with: {'batch_size': 10, 'epochs': 20}
Means: 0.8118811845779419, Stdev: 0.042777153577535476 with: {'batch_size': 10, 'epochs': 30}
Means: 0.6402640243371328, Stdev: 0.12636428620113693 with: {'batch_size': 10, 'epochs': 45}
Means: 0.7953795393308004, Stdev: 0.02034459460670113 with: {'batch_size': 10, 'epochs': 60}
Means: 0.7788778940836588, Stdev: 0.03645332582644608 with: {'batch_size': 10, 'epochs': 80}
Means: 0.7425742745399475, Stdev: 0.0370461016997341 with: {'batch_size': 10, 'epochs': 120}


### 30 epochs increased my score to 81%

In [133]:
def build_model():
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def test_model():
    param_grid = {'batch_size': [10,20,50,60],
                  'epochs': [30,150,200]}
    
    model = KerasClassifier(build_fn=build_model, verbose=False, validation_split=0.10)

    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
    grid_result = grid.fit(X, y)

    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [134]:
test_model()

Best: 0.7854785521825155 using {'batch_size': 20, 'epochs': 150}
Means: 0.5643564363320669, Stdev: 0.1711086648947219 with: {'batch_size': 10, 'epochs': 30}
Means: 0.7524752616882324, Stdev: 0.021388576788767738 with: {'batch_size': 10, 'epochs': 150}
Means: 0.7623762488365173, Stdev: 0.03523787151819811 with: {'batch_size': 10, 'epochs': 200}
Means: 0.5874587496121725, Stdev: 0.15174341516061454 with: {'batch_size': 20, 'epochs': 30}
Means: 0.7854785521825155, Stdev: 0.04148449288410131 with: {'batch_size': 20, 'epochs': 150}
Means: 0.4554455478986104, Stdev: 0.0352378854744143 with: {'batch_size': 20, 'epochs': 200}
Means: 0.6798679927984873, Stdev: 0.13916282954842815 with: {'batch_size': 50, 'epochs': 30}
Means: 0.6897689799467722, Stdev: 0.1543058664364082 with: {'batch_size': 50, 'epochs': 150}
Means: 0.6963696380456289, Stdev: 0.15281632439934778 with: {'batch_size': 50, 'epochs': 200}
Means: 0.6171617209911346, Stdev: 0.12162063284890626 with: {'batch_size': 60, 'epochs': 30}
M